In [ ]:
from pathlib import Path
from zipfile import ZipFile

from rp2 import fetch_file, get_data_path

In [ ]:
hagai_path = get_data_path("hagai_2018")
fetch_file(
    "https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-018-0657-2/MediaObjects/41586_2018_657_MOESM4_ESM.xlsx",
    hagai_path
);

In [ ]:
ae_path = get_data_path("ArrayExpress")
fetch_file(
    "https://www.ebi.ac.uk/arrayexpress/files/E-MTAB-6754/E-MTAB-6754.processed.2.zip",
    ae_path
);

In [ ]:
for zip_path in ae_path.glob("*.zip"):
    extract_path = ae_path.joinpath(zip_path.stem)
    if extract_path.exists():
        continue

    print("Extracting:", zip_path)

    with ZipFile(zip_path, "r") as zip_file:
        zip_file.extractall(extract_path)

In [ ]:
txburst_filenames = (
    "txburstML.py",
    "txburstPL.py",
    "txburstTEST.py",
)

txburst_path = Path("txburst")

for filename in txburst_filenames:
    url = f"https://raw.githubusercontent.com/sandberg-lab/txburst/master/{filename}"
    fetch_file(url, txburst_path)